In [1]:
import os
import sys
import torch as t
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch import Tensor
from torch.utils.data import DataLoader
import numpy as np
import einops
import wandb
import plotly.express as px
from pathlib import Path
import itertools
import random
from IPython.display import display
import wandb
from jaxtyping import Float, Int, Bool, Shaped, jaxtyped
from typing import Union, Optional, Tuple, Callable, Dict
from collections import Counter
import typeguard
from functools import partial
import copy
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import dataclasses
import datasets
from IPython.display import HTML
import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import HookedRootModule, HookPoint
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache
from tqdm.notebook import tqdm
from dataclasses import dataclass
from rich import print as rprint
import pandas as pd
import torch
import matplotlib.pyplot as plt


import circuits.eval_sae_as_classifier as eval_sae
import circuits.analysis as analysis
import circuits.eval_board_reconstruction as eval_board_reconstruction
import circuits.get_eval_results as get_eval_results
import circuits.f1_analysis as f1_analysis
import circuits.utils as utils
import circuits.pipeline_config as pipeline_config
from circuits.dictionary_learning.dictionary import AutoEncoder, GatedAutoEncoder, AutoEncoderNew
import common
import chess_utils
import chess
#from plotly_utils import imshow
#from neel_plotly import scatter, line



device = t.device('cuda' if t.cuda.is_available() else 'cpu')
import pickle
with open('meta.pkl', 'rb') as picklefile:
    meta = pickle.load(picklefile)

/opt/conda/envs/chessgpt/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
autoencoder = common.load_autoencoder(device)
model = common.load_model(device)
dataset = common.get_dataset(device)
TRAIN_TEST_GAME_SPLIT = 2500
rows_to_include = 250

/opt/conda/envs/chessgpt/lib/python3.11/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading statistics aggregation dataset


In [3]:
encoded_inputs_tensor = t.stack([t.tensor(x) for x in dataset['encoded_inputs']]).to(device)
is_check = dataset['board_to_check_state'].squeeze(-1)
is_this_a_dot = (encoded_inputs_tensor == meta['stoi']['.']).to(device)
is_this_a_dot = is_this_a_dot[:rows_to_include]
is_this_a_dot.shape

torch.Size([250, 256])

In [4]:
aggregation_results = common.get_aggregation_results(1000)
formatted_results = common.get_formatted_results(aggregation_results)
features_for_check_state = common.get_true_feature_indices(formatted_results, "board_to_check_state")
features_for_check_state = features_for_check_state.to(device)

In [5]:
#Collect all the activations for the features we care about
relevant_sae_activations = []
def get_activation(name, relevant_features):
    def hook(model, input, output):
        encoded_activations = autoencoder.encode(output[0]) # batch_size x len_seq x n_features_sae
        relevant_sae_activations.append(encoded_activations[:, :, relevant_features])
    return hook

t.set_grad_enabled(False)
activation_handle = model.transformer.h[5].register_forward_hook(get_activation(f"resid_stream_{5}", np.arange(4096)))
num_batches = 50
batch_size = rows_to_include // num_batches
if device == 'cpu':
    num_batches = 1
    batch_size = 1
for i in range(num_batches):
    print(i)
    model(encoded_inputs_tensor[i*batch_size:(i+1)*batch_size])
    t.cuda.empty_cache()
activation_handle.remove()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [6]:
all_relevant_sae_activations = t.concat(relevant_sae_activations, dim=0) # batch_size x len_seq x n_check_features_sae
del relevant_sae_activations
t.cuda.empty_cache()
all_relevant_sae_activations.shape, is_this_a_dot.shape


(torch.Size([250, 256, 4096]), torch.Size([250, 256]))

In [7]:
all_relevant_sae_activations.shape
is_this_a_dot.shape
is_this_a_space = (encoded_inputs_tensor == meta['stoi'][' ']).to(device)
is_this_a_space = is_this_a_space.long()
is_this_a_space = is_this_a_space[:rows_to_include]

In [8]:
def alternate_ones(row):
    # Find indices where there are 1s
    ones_indices = (row == 1).nonzero(as_tuple=True)[0]
    
    # Assign alternating 1 and 2
    for i, idx in enumerate(ones_indices):
        row[idx] = 1 if i % 2 == 0 else 2
    return row

# Apply the function to each row
result_tensor = torch.stack([alternate_ones(row.clone()) for row in is_this_a_space])
is_this_a_first_space = result_tensor==1
is_this_a_second_space = result_tensor==2


In [9]:
activations_on_dot = all_relevant_sae_activations * is_this_a_dot.unsqueeze(-1)
activations_on_first_space = all_relevant_sae_activations * is_this_a_first_space.unsqueeze(-1)
activations_on_second_space = all_relevant_sae_activations * is_this_a_second_space.unsqueeze(-1)

In [10]:
max_activations = all_relevant_sae_activations.max(dim=0).values.max(dim=0).values
del all_relevant_sae_activations
t.cuda.empty_cache()

In [11]:
threshold_multiplier = 0.2
(activations_on_dot > threshold_multiplier * max_activations).sum(dim=0).sum(dim=0).shape

OutOfMemoryError: CUDA out of memory. Tried to allocate 1.95 GiB. GPU 0 has a total capacty of 23.69 GiB of which 1.77 GiB is free. Process 1810083 has 4.29 GiB memory in use. Process 1966692 has 3.36 GiB memory in use. Process 2038806 has 3.45 GiB memory in use. Process 2067979 has 5.27 GiB memory in use. Process 2091698 has 5.53 GiB memory in use. Of the allocated memory 5.19 GiB is allocated by PyTorch, and 39.94 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF